# Utils

In [1]:
import numpy as np
from lib.data.metainfo import MetaInfo
from lib.data.transforms import BaseTransform
import hydra
from lib.utils.config import load_config
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
from lib.visualize.image import image_grid
from torch.nn.functional import l1_loss
import cv2 as cv
from torchvision.transforms import v2

def transform(normal):
    _transform = BaseTransform()
    return _transform(normal).to("cuda")

def plot_images_np(images, size: int = 4):
    if isinstance(images, list):
        _, axes = plt.subplots(1, len(images), figsize=(size, size))
        for ax, image in zip(axes, images):
            ax.imshow(image)
            ax.axis("off")  # Turn off axis
        plt.show()
    else:
        plt.figure(figsize=(size, size))
        plt.imshow(images)
        plt.show()

def plot_images(images, size: int = 4):
    if isinstance(images, list):
        _, axes = plt.subplots(1, len(images), figsize=(size, size))
        for ax, image in zip(axes, images):
            ax.imshow(image.permute(1, 2, 0).detach().cpu().numpy())
            ax.axis("off")  # Turn off axis
        plt.show()
    else:
        plt.figure(figsize=(size, size))
        plt.imshow(images.permute(1, 2, 0).detach().cpu().numpy())
        plt.show()

cfg = load_config("optimize_sketch", ["+dataset=shapenet_chair_4096"])
metainfo = MetaInfo(cfg.data.data_dir)

cfg.model.prior_obj_id = metainfo.obj_ids[0]
cfg.model.loss_ckpt_path = "/home/borth/sketch2shape/checkpoints/latent_siamese_sketch_grayscale_multi_view_256.ckpt"
cfg.model.latent_init = "latent"
cfg.model.retrieval_k = 4
model = hydra.utils.instantiate(cfg.model).to("cuda")

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


# Preprocess the Synthetic Dataset

In [ ]:
from lib.data.transforms import BaseTransform
from lib.render.camera import Camera
from lib.render.mesh import normal_to_grayscale

view_id = 0
transform = BaseTransform(to_image=True, normalize=False)
def normals_to_grayscales(
    normals: np.ndarray,
    azims: list = [],
    elevs: list = [],
):
    grayscales = []
    view_id = 0
    for azim in azims:
        for elev in elevs:
            camera_position = Camera(azim=azim, elev=elev).camera_position()
            grayscale = normal_to_grayscale(normals[0], camera_position)
            grayscales.append(grayscale)
            view_id += 1
    return np.stack(grayscales)

In [ ]:
plt.imshow(grayscale)

In [ ]:
view_id = 0
obj_id = metainfo.obj_ids[0]
for view_id, camera_position in tqdm(enumerate(camera_positions), total=len(camera_positions)):
    normal = transform(metainfo.load_normal(obj_id, f"{view_id:05}")).to("cuda")
    normal = normal.permute(1, 2, 0)
    grayscale = model.deepsdf.normal_to_grayscale(normal, camera_position=camera_position)
    view_id += 1

# Render Chair From the Side

In [ ]:
azim = -89
elev = -20
model.latent = model.deepsdf.lat_vecs.weight[0]
model.deepsdf.create_camera(azim=azim, elev=elev)
model.deepsdf.hparams["surface_eps"] = 1e-03
model.deepsdf.hparams["ambient"] = 0.2
model.deepsdf.hparams["diffuse"] = 0.5
rendered_normal = model.capture_camera_frame("grayscale").detach().cpu().numpy()
print(model.deepsdf.camera_position)
plot_images_np(rendered_normal)

# Phong Shader Inf Far Away

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Set mean and standard deviation
mean = 0
std_dev = 1

# Generate random numbers from the Gaussian distribution
data3 = np.random.normal(90, 22.5, 50)
data2 = np.random.normal(45, 22.5, 50)
data1 = np.random.normal(0, 22.5, 50)
data4 = np.random.normal(-45, 22.5, 50)
data5 = np.random.normal(-90, 22.5, 50)

# data3 = np.random.uniform(90 - 22.5, 90 + 22.5, 50)
# data2 = np.random.uniform(45 - 22.5, 45 + 22.5, 50)
# data1 = np.random.uniform(0 - 22.5, 0 + 22.5, 50)
# data4 = np.random.uniform(-90 - 22.5, -90 + 22.5, 50)
# data5 = np.random.uniform(-45 - 22.5, -45 + 22.5, 50)

data = np.concatenate([data1, data2, data3, data4, data5])

# Create histogram
plt.hist(data, bins=50, density=True, alpha=0.6, color='g')

# Plot Gaussian PDF
# xmin, xmax = plt.xlim()
# x = np.linspace(xmin, xmax, 100)
# p = np.exp(-0.5 * ((x - mean) / std_dev) ** 2) / (std_dev * np.sqrt(2 * np.pi))
# plt.plot(x, p, 'k', linewidth=2)

plt.title('Gaussian distribution with mean = {} and std = {}'.format(mean, std_dev))
plt.xlabel('Value')
plt.ylabel('Frequency')
plt.show()


In [ ]:
model.deepsdf.create_camera(azim=-90, elev=-45)
obj_id = 0
light_direction = model.deepsdf.camera_position
# light_direction = [1.0, 1.0, 1.0]
ambient = 0.3
diffuse = 0.5

metainfo = MetaInfo(data_dir="/home/borth/sketch2shape/data/shapenet_chair_4096")
trans = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize(mean=[0.5], std=[0.5])
])
# normal = trans(metainfo.load_normal(metainfo.obj_ids[obj_id], "00011"))
# normal = trans(metainfo.load_normal(metainfo.obj_ids[obj_id], "00081"))
model.latent = model.deepsdf.lat_vecs.weight[5]
normal = trans(model.capture_camera_frame().detach().cpu().numpy())
mask = normal.sum(0) > 2.95
# normal[0, :, :] *= -1  # flip the shadow so that it looks from every side the same
# normal[:, mask] = 0.0

L = torch.tensor(light_direction.detach().cpu())  #  (3,)
# L = torch.tensor(light_direction)
L = L / torch.norm(L)
image = torch.zeros_like(normal)
image += ambient
image += diffuse * (L[..., None, None] * normal).sum(0)[None, ...]
image[:, mask] = 1
plot_images(image, size=4)

In [ ]:
(L[..., None, None] * normal).sum(0)[None, ...].max()

# Mean Shader

In [ ]:
obj_id = 0
metainfo = MetaInfo(data_dir="/home/borth/sketch2shape/data/shapenet_chair_4096")
trans = v2.Compose([
    v2.ToImage(),
    v2.ToDtype(torch.float32, scale=True),
])
# normal = trans(metainfo.load_normal(metainfo.obj_ids[obj_id], "00081"))
normal = trans(metainfo.load_normal(metainfo.obj_ids[obj_id], "00011"))
mask = normal.sum(0) > 2.95
# normal[2, :, :] *= -1  # flip the shadow
normal[:, mask] = 1
mean = normal.mean(0)
image = torch.stack([mean, mean, mean], dim=0)
plot_images(image, size=4)